# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "./output_data/cities.csv" 

# Read the csv file 
city_data_df = pd.read_csv(output_data_file) # City data is cleaned: humidity > 100 dropped

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,luderitz,-26.6481,15.1594,73.98,46,0,10.20,NaN,1613988724
1,1,liku,-0.7275,119.8922,78.22,80,82,4.43,ID,1613988724
2,2,hermanus,-34.4187,19.2345,78.01,43,0,10.45,ZA,1613988724
3,3,vaini,-21.2000,-175.2000,77.00,78,20,10.36,TO,1613988724
4,4,provideniya,64.3833,-173.3000,-10.68,85,47,2.68,RU,1613988724


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
fig = gmaps.figure(center=(15, 25), zoom_level=2)
locations_df = city_data_df[["Lat", "Lng"]]

# Clean DF, remove NaN and drop date
clean_locations_df = city_data_df[["City","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country"]].dropna(how='any')
humidity_df = city_data_df["Humidity"].astype(float)

# print(clean_locations_df)
# print(humidity_df)

max_humidity = city_data_df['Humidity'].max()

# Create and add heatmap layer
heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=5)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

<h3>Ideal Weather Conditions</h3><ul><li>Max Temperature between 61-82 degrees Fahrenheit</li><li>Humidity below 100%</li><li>Cloudiness less than 80%</li><li>Wind Speed below 30 mph</li></ul>

In [5]:
# narrow down criteria 
ideal_locations_df = clean_locations_df.loc[(clean_locations_df["Max Temp"]>60) & (clean_locations_df["Max Temp"]<83) &
                                   (clean_locations_df["Humidity"]<100) & (clean_locations_df["Cloudiness"]<80) &
                                   (clean_locations_df["Wind Speed"]<30)]
ideal_locations_df = ideal_locations_df.reset_index(drop=True)
ideal_locations_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,hermanus,-34.4187,19.2345,78.01,43,0,10.45,ZA
1,vaini,-21.2000,-175.2000,77.00,78,20,10.36,TO
2,chuy,-33.6971,-53.4616,68.77,95,32,8.66,UY
3,netivot,31.4222,34.5886,66.99,52,30,4.34,IL
4,itoman,26.1247,127.6694,71.01,64,40,3.44,JP
...,...,...,...,...,...,...,...,...
153,ganzhou,28.8184,115.4327,69.55,71,60,3.22,CN
154,esmeraldas,-19.7625,-44.3139,68.00,94,75,3.44,BR
155,san vicente,-26.6167,-54.1333,67.21,84,39,2.30,AR
156,khunti,23.0833,85.2833,82.40,26,3,2.30,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_locations_df

# Add hotel name column 
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,hermanus,-34.4187,19.2345,78.01,43,0,10.45,ZA,
1,vaini,-21.2000,-175.2000,77.00,78,20,10.36,TO,
2,chuy,-33.6971,-53.4616,68.77,95,32,8.66,UY,
3,netivot,31.4222,34.5886,66.99,52,30,4.34,IL,
4,itoman,26.1247,127.6694,71.01,64,40,3.44,JP,
...,...,...,...,...,...,...,...,...,...
153,ganzhou,28.8184,115.4327,69.55,71,60,3.22,CN,
154,esmeraldas,-19.7625,-44.3139,68.00,94,75,3.44,BR,
155,san vicente,-26.6167,-54.1333,67.21,84,39,2.30,AR,
156,khunti,23.0833,85.2833,82.40,26,3,2.30,IN,


In [8]:
# Set the base url to serch
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    city = row["City"] # For printing
    # Hotel location
    lat= row["Lat"]
    lng= row["Lng"]
    
    # add keyword to params dict
    params["location"] = f"{lat}, {lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response["results"]
    
    try:
        closet_hotel = results[0]["name"]
        print(f"Closest hotel is {closet_hotel}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
                
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: hermanus.
Closest hotel is Misty Waves Boutique Hotel.
------------
Retrieving Results for Index 1: vaini.
Closest hotel is Keleti Beach Resort.
------------
Retrieving Results for Index 2: chuy.
Closest hotel is Nuevo Hotel Plaza.
------------
Retrieving Results for Index 3: netivot.
Closest hotel is 'בית החסד - מוסדות מאורות נתן' - מיסודו של הרב נתן בוקובזה זצוק"ל.
------------
Retrieving Results for Index 4: itoman.
Closest hotel is Southern Beach Hotel & Resort Okinawa.
------------
Retrieving Results for Index 5: taltal.
Closest hotel is Residencial Karime.
------------
Retrieving Results for Index 6: viedma.
Closest hotel is Hotel Cristal.
------------
Retrieving Results for Index 7: lima.
Closest hotel is Gran Hotel Bolivar Lima.
------------
Retrieving Results for Index 8: port elizabeth.
Closest hotel is 39 On Nile Guest House.
------------
Retrieving Results for Index 9: ponta do sol.
Closest hotel is Hotel do Campo.
------------
Retrieving Res

Closest hotel is Hostal Rio Miel.
------------
Retrieving Results for Index 86: sao jose da coroa grande.
Closest hotel is Crôa Mares Hotel.
------------
Retrieving Results for Index 87: anqing.
Closest hotel is Hongtian Hotel.
------------
Retrieving Results for Index 88: cap-haitien.
Closest hotel is Habitation Jouissant.
------------
Retrieving Results for Index 89: sur.
Closest hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 90: nedjo.
Closest hotel is Yerosan Hotel.
------------
Retrieving Results for Index 91: galesong.
Closest hotel is Sapriadi Appy.
------------
Retrieving Results for Index 92: camacha.
Closest hotel is Hotel Porto Santo & Spa.
------------
Retrieving Results for Index 93: birjand.
Closest hotel is Birjand Tourist Hotel.
------------
Retrieving Results for Index 94: alofi.
Closest hotel is Taloa Heights.
------------
Retrieving Results for Index 95: cabo san lucas.
Closest hotel is Hotel Tesoro Los Cabos.
------------
Retrieving Results for 

In [14]:
# Drop cities with missing hotel
hotel_df.replace('', np.nan, inplace=True) # Change blank string to nan
hotel_df.dropna(how='any', inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,hermanus,-34.4187,19.2345,78.01,43,0,10.45,ZA,Misty Waves Boutique Hotel
1,vaini,-21.2000,-175.2000,77.00,78,20,10.36,TO,Keleti Beach Resort
2,chuy,-33.6971,-53.4616,68.77,95,32,8.66,UY,Nuevo Hotel Plaza
3,netivot,31.4222,34.5886,66.99,52,30,4.34,IL,'בית החסד - מוסדות מאורות נתן' - מיסודו של הרב...
4,itoman,26.1247,127.6694,71.01,64,40,3.44,JP,Southern Beach Hotel & Resort Okinawa
...,...,...,...,...,...,...,...,...,...
152,general roca,-39.0333,-67.5833,60.80,55,0,5.75,AR,Hotel y Casino del Río General Roca
153,ganzhou,28.8184,115.4327,69.55,71,60,3.22,CN,Laotou Farm Restaurant Hostel
154,esmeraldas,-19.7625,-44.3139,68.00,94,75,3.44,BR,Bom Será - Marcinho e Fernanda
155,san vicente,-26.6167,-54.1333,67.21,84,39,2.30,AR,Complejo Turístico Farelluk


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))